<a href="https://colab.research.google.com/github/CoolerKula/Hello-world/blob/master/The_Linear_Regression_Model_Project_Bernard_Kula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Notebook: The Linear Regression Model

## 1. Introduction

We started by building intuition for model based learning, explored how the linear regression model worked, understood how the two different approaches to model fitting worked, and some techniques for cleaning, transforming, and selecting features. In this project, you can practice what you learned by exploring ways to improve the models we built.

You'll work with housing data for the city of Ames, Iowa, United States from 2006 to 2010. You can also read about the different columns in the data [here](https://s3.amazonaws.com/dq-content/307/data_description.txt).

Let's start by setting up a pipeline of functions that will let us quickly iterate on different models.

**Tasks**

1. Import pandas, matplotlib, and numpy into the environment. Import the classes you need from scikit-learn as well.
2. Read `AmesHousing.tsv` () into a pandas data frame.
3. For the following functions, we recommend creating them in the first few cells in the notebook. This way, you can add cells to the end of the notebook to do experiments and update the functions in these cells.
* Create a function named `transform_features()` that, for now, just returns the train data frame.
* Create a function named `select_features()` that, for now, just returns the Gr Liv Area and SalePrice columns from the train data frame.
* Create a function named `train_and_test()` that, for now:

1. Selects the first 1460 rows from from data and assign to train.
2. Selects the remaining rows from data and assign to test.
3. Trains a model using all numerical columns except the SalePrice column (the target column) from the data frame returned from `select_features()`
4. Tests the model on the test set and returns the `RMSE` value.

In [81]:
# Import pandas, matplotlib, and numpy into the environment. Import the classes you need from scikit-learn as well
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd
pd.options.display.max_columns = 999
import matplotlib as mpl
from sklearn import linear_model
from sklearn.model_selection import KFold

In [82]:
# Read AmesHousing.tsv () into a pandas data frame
data = pd.read_csv('https://bit.ly/3boZCX4', sep='\t')
data

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,GasA,Fa,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,None,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,GasA,Ex,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,2926,923275080,80,RL,37.0,7937,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,Norm,Norm,1Fam,SLvl,6,6,1984,1984,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,CBlock,TA,TA,Av,GLQ,819.0,Unf,0.0,184.0,1003.0,GasA,TA,Y,SBrkr,1003,0,0,1003,1.0,0.0,1,0,3,1,TA,6,Typ,0,NaN,Detchd,1984.0,Unf,2.0,588.0,TA,TA,Y,120,0,0,0,0,0,NaN,GdPrv,NaN,0,3,2006,WD,Normal,142500
2926,2927,923276100,20,RL,NaN,8885,Pave,NaN,IR1,Low,AllPub,Inside,Mod,Mitchel,Norm,Norm,1Fam,1Story,5,5,1983,1983,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,CBlock,Gd,TA,Av,BLQ,301.0,ALQ,324.0,239.0,864.0,GasA,TA,Y,SBrkr,902,0,0,902,1.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1983.0,Unf,2.0,484.0,TA,TA,Y,164,0,0,0,0,0,NaN,MnPrv,NaN,0,6,2006,WD,Normal,131000
2927,2928,923400125,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,SFoyer,5,5,1992,1992,Gable,CompShg,HdBoard,Wd Shng,None,0.0,TA,TA,PConc,Gd,TA,Av,GLQ,337.0,Unf,0.0,575.0,912.0,GasA,TA,Y,SBrkr,970,0,0,970,0.0,1.0,1,

In [83]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 82 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order            2930 non-null   int64  
 1   PID              2930 non-null   int64  
 2   MS SubClass      2930 non-null   int64  
 3   MS Zoning        2930 non-null   object 
 4   Lot Frontage     2440 non-null   float64
 5   Lot Area         2930 non-null   int64  
 6   Street           2930 non-null   object 
 7   Alley            198 non-null    object 
 8   Lot Shape        2930 non-null   object 
 9   Land Contour     2930 non-null   object 
 10  Utilities        2930 non-null   object 
 11  Lot Config       2930 non-null   object 
 12  Land Slope       2930 non-null   object 
 13  Neighborhood     2930 non-null   object 
 14  Condition 1      2930 non-null   object 
 15  Condition 2      2930 non-null   object 
 16  Bldg Type        2930 non-null   object 
 17  House Style   

In [84]:
# Create a function named transform_features() that, for now, just returns the train data frame
def transform_features(data):
    return data

In [85]:
# Create a function named select_features() that, for now, 
# just returns the Gr Liv Area and SalePrice columns from the train data frame
def select_features(data):
    return data[['Gr Liv Area', 'SalePrice']]

In [86]:
# Create a function named train_and_test() that, for now:
# Selects the first 1460 rows from from data and assign to train.
# Selects the remaining rows from data and assign to test.
# Trains a model using all numerical columns except the SalePrice column 
# (the target column) from the data frame returned from select_features()
# Tests the model on the test set and returns the RMSE value.

def train_and_test(data):
    train = data[0:1460]
    test = data[1460:]
    
    # Using pd.dataframe.select_dtypes to only includes 
    # columns with certain datatypes.
    train_numeric = train.select_dtypes(include = ['float','integer'])
    test_numeric = test.select_dtypes(include = ['float', 'integer'])
    
    # Using pd.drop to drop a value
    features = train_numeric.columns.drop('SalePrice')
    
    # Using target column for solution extensibility
    target = 'SalePrice'
    
    # Creating Linear Regression instance:
    lr = LinearRegression()
    lr.fit(train[features], train[target])
    
    # Predicting on test set
    predictions = lr.predict(test[features])
    
    # Calculating errors (MSE and RMSE)
    mse = mean_squared_error(test[target], predictions)
    rmse = np.sqrt(mse)
    #rmse = mse**(1/2) Alternative square-root calculation

    return rmse

# Using above functions to process dataframe and return RMSE
transform_df = transform_features(data)
features_df = select_features(transform_df)
rmse = train_and_test(features_df)

# Printing rmse from above
print(rmse)

57088.25161263909


## 2. Feature Engineering

Let's now start removing features with many missing values, diving deeper into potential categorical features, and transforming text and numerical columns. Update `transform_features()` so that any column from the data frame with more than 25% (or another cutoff value) missing values is dropped. You also need to remove any columns that leak information about the sale (e.g. like the year the sale happened). In general, the goal of this function is to:

* remove features that we don't want to use in the model, just based on the number of missing values or data leakage.
* transform features into the proper format (numerical to categorical, scaling numerical, filling in missing values, etc).
* create new features by combining other features.

Next, you need to get more familiar with the remaining columns by reading the data documentation for each column, determining what transformations are necessary (if any), and more. As we mentioned earlier, succeeding in predictive modeling (and competitions like Kaggle) is highly dependent on the quality of features the model has. Libraries like scikit-learn have made it quick and easy to simply try and tweak many different models, but cleaning, selecting, and transforming features are still more of an art that requires a bit of human ingenuity.

**Tasks**

1. As we mentioned earlier, we recommend adding some cells to explore and experiment with different features (before rewriting these functions).

2. The `transform_features()` function shouldn't modify the train data frame and instead return a new one entirely. This way, we can keep using train in the experimentation cells.

3. Which columns contain less than 5% missing values?
* For numerical columns that meet this criteria, let's fill in the missing values using the most popular value for that column.

4. What new features can we create, that better capture the information in some of the features?
* An example of this would be the `years_until_remod` feature we created in the last lesson.

5. Which columns need to be dropped for other reasons?
* Which columns aren't useful for machine learning?
* Which columns leak data about the final sale?

In [87]:
# Find number of missing values, sorted by most missing
missing_count = data.isnull().sum().sort_values(ascending = False)
missing_count.head()

Pool QC         2917
Misc Feature    2824
Alley           2732
Fence           2358
Fireplace Qu    1422
dtype: int64

In [88]:
# Create variable for columns with more than 5% missing
high_missing = missing_count[(missing_count/len(data) > .05)]
print(high_missing.sort_values(ascending=False))

# Dropping these columns from the dataframe
# In this case index is returning the label, which we are dropping
# axis = 1 refers to the column axis in the dataframe
data = data.drop(high_missing.index, axis = 1)

Pool QC          2917
Misc Feature     2824
Alley            2732
Fence            2358
Fireplace Qu     1422
Lot Frontage      490
Garage Finish     159
Garage Cond       159
Garage Yr Blt     159
Garage Qual       159
Garage Type       157
dtype: int64


In [89]:
print(data.head())

   Order        PID  MS SubClass MS Zoning  Lot Area Street Lot Shape  \
0      1  526301100           20        RL     31770   Pave       IR1   
1      2  526350040           20        RH     11622   Pave       Reg   
2      3  526351010           20        RL     14267   Pave       IR1   
3      4  526353030           20        RL     11160   Pave       Reg   
4      5  527105010           60        RL     13830   Pave       IR1   

  Land Contour Utilities Lot Config Land Slope Neighborhood Condition 1  \
0          Lvl    AllPub     Corner        Gtl        NAmes        Norm   
1          Lvl    AllPub     Inside        Gtl        NAmes       Feedr   
2          Lvl    AllPub     Corner        Gtl        NAmes        Norm   
3          Lvl    AllPub     Corner        Gtl        NAmes        Norm   
4          Lvl    AllPub     Inside        Gtl      Gilbert        Norm   

  Condition 2 Bldg Type House Style  Overall Qual  Overall Cond  Year Built  \
0        Norm      1Fam      1S

In [90]:
# Dropping text columns with more than 1 missing value

# Finding text columns with missing values
text_mv_counts = data.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)

# Filter Series to columns containing *any* missing values
drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]

data = data.drop(drop_missing_cols_2.index, axis=1)

In [91]:
# Filling in missing numeric values with most popular for that column

# Finding missing numeric values
numeric_missing = df.select_dtypes(include=['integer', 'float']).isnull().sum().sort_values(ascending = False)

# Finding columns between 1 missing value and 5% missing values
fixable_numeric_cols = numeric_missing[(numeric_missing < len(df)/20) & (numeric_missing > 0)].sort_values(ascending = False)
fixable_numeric_cols

Mas Vnr Area      23
Bsmt Full Bath     2
Bsmt Half Bath     2
Garage Area        1
Total Bsmt SF      1
Bsmt Unf SF        1
BsmtFin SF 2       1
BsmtFin SF 1       1
Garage Cars        1
dtype: int64

In [92]:
# Compute the most common value for each column in `fixable_numeric_missing_cols`.
replacement_values_dict = data[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
replacement_values_dict

{'Bsmt Full Bath': 0.0,
 'Bsmt Half Bath': 0.0,
 'Bsmt Unf SF': 0.0,
 'BsmtFin SF 1': 0.0,
 'BsmtFin SF 2': 0.0,
 'Garage Area': 0.0,
 'Garage Cars': 2.0,
 'Mas Vnr Area': 0.0,
 'Total Bsmt SF': 0.0}

In [93]:
# Fill values in fixable numeric missing columns from above
data = data.fillna(replacement_values_dict)

In [94]:
# Check again for missing values
data.isnull().sum().value_counts()

0    64
dtype: int64

In [95]:
# Creating feature for age of home at sale
home_age = data['Yr Sold'] - data['Year Built']

# Checking for unexpected (negative values)
home_age[home_age < 0]

2180   -1
dtype: int64

In [96]:
# Creating features for age of remodel at sale
remod_age = data['Yr Sold'] - data['Year Remod/Add']

# Checking for negative values
remod_age[remod_age < 0]

1702   -1
2180   -2
2181   -1
dtype: int64

In [97]:
# Adding the above new features to the dataframe

# Adding home age feature
data['home_age'] = years_sold

# Adding remodel age feature
data['remod_age'] = remod_age

# Drop rows with negative values for both of these new features
data = data.drop([1702, 2180, 2181], axis=0)

# Drop original year columns
data = data.drop(["Year Built", "Year Remod/Add"], axis = 1)

NameError: ignored

In [ ]:
# Drop columns that aren't useful for ML
data = data.drop(["PID", "Order"], axis=1)

# Drop columns that leak info about the final sale
data = data.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)

In [ ]:
# Updating the transform_features function
# See above cells for commentary on each of these updates

def transform_features(data):
    missing_count = data.isnull().sum().sort_values(ascending = False)
    high_missing = missing_count[(missing_count/len(data) > .05)]
    data = data.drop(high_missing.index, axis = 1)
    
    text_mv_counts = data.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]
    data = data.drop(drop_missing_cols_2.index, axis=1)
    
    numeric_missing = data.select_dtypes(include=['integer', 'float']).isnull().sum().sort_values(ascending = False)
    fixable_numeric_cols = numeric_missing[(numeric_missing < len(data)/20) & (numeric_missing > 0)].sort_values(ascending = False)
    replacement_values_dict = data[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
    data = data.fillna(replacement_values_dict)
    
    home_age = data['Yr Sold'] - data['Year Built']
    
    remod_age = data['Yr Sold'] - data['Year Remod/Add']
    
    data['home_age'] = years_sold


    data['remod_age'] = remod_age

    data = data.drop([1702, 2180, 2181], axis=0)

    data = data.drop(["Year Built", "Year Remod/Add"], axis = 1)
 
    data = data.drop(["PID", "Order"], axis=1)

    data = data.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)
        
    return data

def select_features(data):
    return data[['Gr Liv Area', 'SalePrice']]

def train_and_test(data):
    train = data[0:1460]
    test = data[1460:]
    
    # Using pd.dataframe.select_dtypes to only includes 
    # columns with certain datatypes.
    train_numeric = train.select_dtypes(include = ['float','integer'])
    test_numeric = test.select_dtypes(include = ['float', 'integer'])
    
    # Using pd.drop to drop a value
    features = train_numeric.columns.drop('SalePrice')
    
    # Using target column for solution extensibility
    target = 'SalePrice'
    
    # Creating Linear Regression instance:
    lr = LinearRegression()
    lr.fit(train[features], train[target])
    
    # Predicting on test set
    predictions = lr.predict(test[features])
    
    # Calculating errors (MSE and RMSE)
    mse = mean_squared_error(test[target], predictions)
    rmse = np.sqrt(mse)
    #rmse = mse**(1/2) Alternative square-root calculation

    return rmse

# Reading in fresh dataframe
data = pd.read_csv("AmesHousing.tsv", delimiter="\t")

# Using above functions to process dataframe and return RMSE
transform_df = transform_features(data)
features_df = select_features(transform_df)
rmse = train_and_test(features_df)

# Printing rmse from above
print(rmse)

In [ ]:
numerical_train.describe()

In [ ]:
numerical_train.isnull().sum()

In [ ]:
# Your code goes here
def transform_features():
    return (train)

## 3. Feature Selection

Now that we have cleaned and transformed a lot of the features in the data set, it's time to move on to feature selection for numerical features.

**Tasks**

1. Generate a correlation heatmap matrix of the numerical features in the training data set.
* Which features correlate strongly with our target column, `SalePrice`?
* Calculate the correlation coefficients for the columns that seem to correlate well with `SalePrice`. Because we have a pipeline in place, it's easy to try different features and see which features result in a better cross validation score.

2. Which columns in the data frame should be converted to the categorical data type? All of the columns that can be categorized as nominal variables are candidates for being converted to categorical. Here are some other things you should think about:
* If a categorical column has hundreds of unique values (or categories), should you keep it? When you dummy code this column, hundreds of columns will need to be added back to the data frame.
* Which categorical columns have a few unique values but more than 95% of the values in the column belong to a specific category? This would be similar to a low variance numerical feature (no variability in the data for the model to capture).

3. Which columns are currently numerical but need to be encoded as categorical instead (because the numbers don't have any semantic meaning)?

4. What are some ways we can explore which categorical columns "correlate" well with `SalePrice`?

5. Update the logic for the `select_features()` function. This function should take in the new, modified train and test data frames that were returned from `transform_features()`.

In [ ]:
# Your code goes here


## 4. Train and Test

Now for the final part of the pipeline, training and testing. When iterating on different features, using simple validation is a good idea. Let's add a parameter named `k` that controls the type of cross validation that occurs.

**Tasks**

1. The optional `k` parameter should accept integer values, with a default value of `0`.

2. When `k` equals `0`, perform holdout validation (what we already implemented):

* Select the first `1460` rows and assign to `train`.
* Select the remaining rows and assign to test.
* Train on `train` and `test` on test.
* Compute the `RMSE` and return.

3. When k equals 1, perform simple cross validation:

* Shuffle the ordering of the rows in the data frame.
* Select the first 1460 rows and assign to `fold_one`.
* Select the remaining rows and assign to `fold_two`.
* Train on `fold_one` and test on `fold_two`.
* Train on `fold_two` and test on `fold_one`.
* Compute the average RMSE and return.

4. When `k` is greater than `0`, implement k-fold cross validation using `k` folds:

* Perform `k-fold` cross validation using k folds.
* Calculate the average `RMSE` value and return this value.

In [ ]:
# Your code goes here

## 5. Next Steps

That's it for the guided steps. Here's some potenial next steps that you can take:

1. Continue iteration on feature engineering:
* Research some other approaches to feature engineering online around housing data.
* Visit the Kaggle kernels [page](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/kernels) page for this dataset to see approaches others took.

2. Improve your feature selection:
* Research ways of doing feature selection better with categorical columns (something we didn't cover in this particular course).